# Accessing data from reconstructed vertices

In this tutorial, we will open a file with reconstructed vertices and inspect the data structure

First, let us set up the FEDRA configuration as usual

In [ ]:
%jsroot on

In [ ]:
TString fedrapath(gSystem->Getenv("FEDRA_ROOT"));
gROOT->ProcessLine((TString(".x ")+fedrapath+TString("/macros/rootlogon.C")).Data())

# Open the file and getting the TTree

The TTree is named "vtx".

We will read it with the RDataFrame class, https://root.cern/doc/master/classROOT_1_1RDataFrame.html,

which allows to easily explain its structure

## Vertex Tree Data format

The Tree contains the following information
Scalar branches (i.e. one number for vertex):

* n: vertex molteplicity (accessed with N());

* vx,vy,vz: coordinates of reconstructed vertex (accessed with VX(), VY(), VZ());

* vCOV: covariance matrix of vertex position (stored as TMatrixD)

* meanvx,meanvy,meanvz: mean point of starting tracks (accessed with X(), Y(),Z());

* maxaperture; maximum angle between tracks in the vertex (computed with MaxAperture ());

* probability: probability of vertex from fit (accessed with V()->prob());

* flag: a flag providing information about vertex topology (accessed with Flag()):
        flag 0: neutral (tracks' starts attached only
        flag 1: charge (tracks' ends&starts attached
        flag 2: back neutral (tracks' ends attached only

    The following flags are similar to the previous one, but they add the information that vertex has common tracks to at least another vertex (if LinkedVertexes () has been called)
        flag 3: neutral, linked (has common tracks with other vertex, only if LinkedVertexes () has been called)
        flag 4: charge, linked
        flag 5: back neutral, linked
        flag -10: discarded vertices (all tracks were associated to higher rank vertices).

In [ ]:
TString filepath(
//"/eos/user/f/falicant/handson_FEDRA/b000021/trackfiles/rootfiles/47_41/cell_9_9/vertextree.root");    
"/eos/experiment/sndlhc/emulsionData/2022/emureco_Napoli/RUN3/b000044/trackfiles/rootfiles/57_1/cell_7_4/vertextree.root");
TFile *vertexfile = TFile::Open(filepath);
TTree *vertextree = (TTree*) vertexfile->Get("vtx")


In [ ]:
using namespace ROOT;

In [ ]:
RDataFrame df(*vertextree); //loading a dataframe with tracks info

In [ ]:
//getting n, flag and probability
auto hn = df.Histo1D<int>("n"); //like TTree->Draw(), RDataFrame can infer range from data
auto hflag = df.Histo1D<int>("flag");
auto hprob = df.Histo1D({"hprob","Probability",110,0.,1.1},"probability"); //or we can provide it


In [ ]:
//draw histograms
TCanvas *cprob= new TCanvas();
hprob->DrawClone();
cprob->Draw();

In [ ]:
//draw histograms
TCanvas *cn= new TCanvas();
hn->DrawClone();
cn->Draw();

In [ ]:
TCanvas *cflag= new TCanvas();
hflag->DrawClone();
cflag->Draw();

Array branches (i.e. track properties):

* nseg[itrk]: number of found segments for each track (accessed with track->N());
* npl[itrk]: number of expeceted segments for each track (accessed with track->Npl());
* incoming[itrk]: tell us if track ends or start at vertex (accessed with GetVTa (itrk)->ZPos()). 1-track start, 0-track end connect to the vertex
* nholes[itrk]; number of holes for each track (accessed with track->N0());
* maxgap[itrk]: maximum gap between segments of each track (accessed with track->CheckMaxGap());
* impactparameter[itrk]: impact parameter of track with respect to vertex (accessed with vertex->GetVTa(itrk)->Imp());

Arrays of FEDRA objects (as from linked_tracks.root format)

* t.: TClonesArray of EdbSegP with tracks information;
* s: TClonesArray of EdbSegP with segments information for each track;
* sf: TClonesArray of EdbSegP with fitted segments information for each track;

The following branches are to be used only in vertices from MC simulations, no sense in real data:

 * MCEventID[itrk]: true MCEventID of each track (accessed with track->MCEvt());
 * MCTrackID[itrk]: true MCTrackID of each track (accessed with track->MCTrack());
 * MCTrackPdgCode[itrk]: true MCTrack PdgCode of each track (accessed with track->Vid(0));
 * MCMotherID[itrk]: true MCMotherID of each track (accessed with track->Aid(0));

In [ ]:
const int from_plate = 57; //this sets our reference system. Which is the last plate, which is the first?
const int to_plate = 1;

In [ ]:
auto df1 = df.Define("t_Plate",Form("%d-t.ePID",from_plate));

In [ ]:
auto mydisplay = df1.Filter("flag==4 && n>4").Display({"vID","vz","n","t.eZ","t_Plate","incoming"},10);
mydisplay->Print();

In [ ]:
auto mydisplay = df1.Filter("flag==3 && n>4").Display({"vID","vz","n","t.eZ","t_Plate","t.ePID","incoming"},10);
mydisplay->Print();

## Adding fill factor

We often need to compute additional branches, starting from the ones in the reconstructed TTree.

We show a quick example with the fill factor, a variable is used to inspect how well the tracks were reconstructed.

The fill factor is defined as the ratio between the number of filled films **nseg** (i.e., where the track has segments) and the number of available films **nava**

We can compute it and add it to our dataframe

In [ ]:
auto df2 = df1.Define("nava",Form("%d - %d - t_Plate + 2",from_plate, to_plate)).Define("ff","(RVec<float>)nseg/nava");

In [ ]:
auto hff = df2.Histo1D("ff");

In [ ]:
TCanvas *cff = new TCanvas();
hff->DrawClone();
cff->Draw();